# Model Export, Import, and Simulation

This notebook walks through the fundamentals of interfacing with the application:

1. Construct [Lou et al.'s Allosteric Model](https://doi.org/10.1038/nature03568) of synaptic vesicle fusion and save it in the expected JSON format.
2. Import the model as a MarkovModel object, check its attributes.
3. Simulate the model under a variety of [Ca<sup>2+</sup>] stimuli.

# 1. Build and Export Model

Models are stored in standard JSON format. There are three required fields to fully specify a model with N states:
* 'rate_constants': NxN array. Element (i,j) gives the rate constants for transitioning from state i to j.
* 'stim_dependence': NxN array. Element (i,j) indicates whether the i -> j transition is stimulus dependent.
* 'initial_condition': 1xN array. Probability of starting in each state.

Any number of additional metadata items can be included. The following are optional but expected:
* 'name': str. Model name. Default: 'Unnamed'
* 'time_units': str. Units of time (for rate constants). Default: 'ms (assumed)'
* 'stim_units': str. Units of stimulus (for rate constants). Default: 'uM (assumed)'
* 'state_names': list of strings. Names for each state. Default: ['S0','S1',...,'SN']

In [1]:
import numpy as np
import os
import model

# Parameters
kon = 0.1
koff = 4
b = 0.5
f = 31.3
i_plus = 2e-7
replenishment = 0

# Define model
rate_constants = np.array([\
    [0, 5*kon, 0, 0, 0, 0, i_plus*f**0],\
    [1*koff*b**0, 0, 4*kon, 0, 0, 0, i_plus*f**1],\
    [0, 2*koff*b**1, 0, 3*kon, 0, 0, i_plus*f**2],\
    [0, 0, 3*koff*b**2, 0, 2*kon, 0, i_plus*f**3],\
    [0, 0, 0, 4*koff*b**3, 0, 1*kon, i_plus*f**4],\
    [0, 0, 0, 0, 5*koff*b**4, 0, i_plus*f**5],\
    [replenishment, 0, 0, 0, 0, 0, 0]\
    ], dtype=np.float64)

stim_dependence = np.array([\
    [0, 1, 0, 0, 0, 0, 0],\
    [0, 0, 1, 0, 0, 0, 0],\
    [0, 0, 0, 1, 0, 0, 0],\
    [0, 0, 0, 0, 1, 0, 0],\
    [0, 0, 0, 0, 0, 1, 0],\
    [0, 0, 0, 0, 0, 0, 0],\
    [0, 0, 0, 0, 0, 0, 0]
    ], dtype=int)

initial_condition = np.array([\
    [1, 0, 0, 0, 0, 0, 0]
    ], dtype=np.float64)

# Add optional metadata
name = 'allosteric_model'
state_names = ['S0','S1','S2','S3','S4','S5','Fused']
time_units = 'ms'
stim_units = 'uM'
reference = 'Lou,X., Scheuss,V., & Schneggenburger,R. Allosteric modulation of the presynaptic Ca2+ sensor for vesicle fusion. Nature. 435, 497-501 (2005)'
doi = 'https://doi.org/10.1038/nature03568'

# Create dictionary of data and metadata
data = {\
    'name' : name,\
    'rate_constants': rate_constants.tolist(),\
    'stim_dependence': stim_dependence.tolist(),\
    'initial_condition': initial_condition.tolist(),\
    'time_units': time_units,\
    'stim_units': stim_units,\
    'state_names': state_names,\
    'reference': reference,\
    'doi': doi\
    }

# Save as JSON
filename = os.path.join('models',data['name']+'.json')
model.export_model(data, filename)

# 2. Import and Simulate Model

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
filename = os.path.join('models',data['name']+'.json')
allosteric_model = model.import_model(filename)
print(allosteric_model)

allosteric_model: 7-state Markov chain


Simulations can be specified as a paired time / stimulus tuple in one of three ways:

**Constant stimulus**: ([t_end], [stim], n_points=1000)
- t_end: dbl. Simulation end time.
- stim: dbl. Constant stimulus value.
- n_points: int. Number of values to return. Default: 1000.

**1D timeseries stimulus**: ([time], [stim])
- time: 1D array (dbl). Time values.
- stim: 1D array (dbl). Corresponding stimulus values. e.g. a single active zone point.

**3D timeseries stimulus**: ([time], [stim])
- time: 1D array (dbl). Time values.
- stim: 3D array (dbl). len(time) x A1 x A2 where A1 and A1 are the dimensions of the area to simulate. e.g. the entire active zone.

## 2.1. Simulate Constant [Ca<sup>2+</sup>] Stimulus

In [4]:
print(f"Number of simulations completed: {len(allosteric_model.simulations)}")

# Specify simulation
t_end = 1 # ms
ca = 50 # uM
n_points = 1000
simulations = [([t_end], [ca], n_points)]

# Run simulation
allosteric_model.run_simulations(simulations)
print(f"Number of simulations completed: {len(allosteric_model.simulations)}")
print(f"Description of simulations completed:")
for sim_idx, simulation in enumerate(allosteric_model.simulations):
    print(f"  {sim_idx+1}. {simulation['description']}")

Number of simulations completed: 0
Number of simulations completed: 1
Description of simulations completed:
  1. Constant stimulation of 50 uM for 1 ms


In [5]:
time = np.linspace(0,t_end,n_points)
time_label = f"Time ({allosteric_model.metadata['time_units']})"
columns = [time_label] + allosteric_model.metadata['state_names'][:]
data = np.column_stack((time, allosteric_model.simulations[0]['states']))
results = pd.DataFrame(data, columns=columns)
print(results.head())

   Time (ms)        S0        S1        S2        S3            S4  \
0   0.000000  1.000000  0.000000  0.000000  0.000000  0.000000e+00   
1   0.001001  0.975334  0.024419  0.000245  0.000001  3.160883e-09   
2   0.002002  0.951373  0.047660  0.000958  0.000010  4.860190e-08   
3   0.003003  0.928093  0.069769  0.002107  0.000032  2.410146e-07   
4   0.004004  0.905473  0.090791  0.003661  0.000074  7.484875e-07   

             S5         Fused  
0  0.000000e+00  0.000000e+00  
1  3.435321e-12  2.930075e-10  
2  9.930296e-11  8.546805e-10  
3  7.314997e-10  1.849902e-09  
4  3.022677e-09  3.508059e-09  


In [6]:
fig = px.line(results, x=time_label, y=allosteric_model.metadata['state_names'])
fig.update_layout(plot_bgcolor='white')
fig.update_xaxes(ticks='outside', linecolor='black')
fig.update_yaxes(title='Probability', range=[0,1], ticks='outside', linecolor='black')
fig.show()

## 2.2. Simulate 1D [Ca<sup>2+</sup>] Timeseries

Multiple simulations can be requested in one batch by providing a list of stimulus tuples. Here we run three simulations:
1. A constant timeseries (results should be identical to the example above).
2. A linearly increasing [Ca<sup>2+</sup>] trace.
3. An AP-like [Ca<sup>2+</sup>] trace loaded from saved data.

Note that new simulation results are appended to the existing list of results.

In [7]:
# Use the same specifications as before
t_end = 1 # ms
ca = 50 # uM
n_points = 1000

# Constant timeseries
time = np.linspace(0, t_end, n_points)
stimulus = [ca] * len(time)
simulations = [(time, stimulus)]

# Linear ramp
time = np.linspace(0, t_end, n_points)
stimulus = 2 * ca * time
simulations.append((time, stimulus))

# AP-like trace
ca_filepath = os.path.join('stimuli', 'AP_trace.csv')
ca_trace = pd.read_csv(ca_filepath)
simulations.append((ca_trace['Time (ms)'].to_numpy(), ca_trace['Ca (uM)'].to_numpy()))

# Run simulations
print(f"Simulations already completed: {len(allosteric_model.simulations)}")
print(f"Simulations requested: {len(simulations)}")
print(f"Running...")
allosteric_model.run_simulations(simulations)
print(f"New total number of simulations completed: {len(allosteric_model.simulations)}")
print(f"Description of simulations completed:")
for sim_idx, simulation in enumerate(allosteric_model.simulations):
    print(f"  {sim_idx+1}. {simulation['description']}")

Simulations already completed: 1
Simulations requested: 3
Running...
New total number of simulations completed: 4
Description of simulations completed:
  1. Constant stimulation of 50 uM for 1 ms
  2. 1D trace lasting 1.0 ms
  3. 1D trace lasting 1.0 ms
  4. 1D trace lasting 5.0 ms


In [8]:
colours = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

fig = make_subplots(rows=2, cols=3,\
              subplot_titles=('[Ca2+] = 50 uM', 'Linear Ramp', 'AP-like Trace', '', '', ''),\
              vertical_spacing=0.1, horizontal_spacing=0.1, row_heights=[0.3, 0.7])

for sim_idx, simulation in enumerate(allosteric_model.simulations[1:4]):
       time = simulation['simulation'][0]
       stim = simulation['simulation'][1]
       results = pd.DataFrame(np.column_stack((time, simulation['states'])), columns=columns)
       fig.add_trace(go.Scatter(x=time, y=stim, mode='lines', line=dict(color='black'), showlegend=False), row=1, col=sim_idx+1)
       for state_idx, state in enumerate(allosteric_model.metadata['state_names']):
             fig.add_trace(go.Scatter(x=time, y=results[state], mode='lines', line=dict(color=colours[state_idx]), name=state, showlegend=True if sim_idx==0 else False), row=2, col=sim_idx+1)

for col in range(1,4):
       fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False, showline=False, row=1, col=col)
       fig.update_xaxes(title='Time (ms)', ticks='outside', showline=True, linecolor='black', row=2, col=col)
       fig.update_yaxes(title='[Ca2+] (uM)', showgrid=False, ticks='outside', showline=True, linecolor='black', row=1, col=col)
       fig.update_yaxes(title='Probability', range=[0,1], ticks='outside', showline=True, linecolor='black', row=2, col=col)

fig.update_yaxes(tickvals=[50], showline=False, row=1, col=1)
fig.update_layout(plot_bgcolor='white')

fig.show()

## 2.3. Simulate 3D [Ca<sup>2+</sup>] Stimulus

This stimulus specifies the [Ca<sup>2+</sup>] across a 2D region (e.g. an active zone) over multiple frames. Here we simulate two regional stimuli:
1. A 3 x 3 grid in which the centre pixel is fixed at 50 uM (for validation)
2. A 2D Gaussian linearly amplified in time

In [9]:
def gaussuian_filter(kernel_size, sigma=1, muu=0):
 
    # Initializing value of x,y as grid of kernel size
    # in the range of kernel size
 
    x, y = np.meshgrid(np.linspace(-1, 1, kernel_size),
                       np.linspace(-1, 1, kernel_size))
    dst = np.sqrt(x**2+y**2)
 
    # lower normal part of gaussian
    normal = 1/(2 * np.pi * sigma**2)
 
    # Calculating Gaussian filter
    gauss = np.exp(-((dst-muu)**2 / (2.0 * sigma**2))) * normal

    return gauss

In [10]:
# Use the same specifications as before
t_end = 1 # ms
ca = 50 # uM
n_points = 1000

# Create 3x3 stimulus with one pixel fixed at 50 uM
time = np.linspace(0,t_end, n_points)
stimulus = np.asarray([[[0,0,0],[0,ca,0],[0,0,0]]] * len(time))
simulations = [(time, stimulus)]

# Create Gaussian stimulus
kernel_size = 50
time = np.linspace(0,t_end, n_points)
ca_max = 5e2 # uM
stimulus = np.asarray([gaussuian_filter(kernel_size, sigma=0.5, muu=0) * ca for ca in np.linspace(0, ca_max, n_points)])
simulations.append((time, stimulus))

# Run simulations
print(f"Simulations already completed: {len(allosteric_model.simulations)}")
print(f"Simulations requested: {len(simulations)}")
print(f"Running...")
allosteric_model.run_simulations(simulations)
print(f"New total number of simulations completed: {len(allosteric_model.simulations)}")
print(f"Description of simulations completed:")
for sim_idx, simulation in enumerate(allosteric_model.simulations):
    print(f"  {sim_idx+1}. {simulation['description']}")

Simulations already completed: 4
Simulations requested: 2
Running...
New total number of simulations completed: 6
Description of simulations completed:
  1. Constant stimulation of 50 uM for 1 ms
  2. 1D trace lasting 1.0 ms
  3. 1D trace lasting 1.0 ms
  4. 1D trace lasting 5.0 ms
  5. 3D timeseries lasting 1.0 ms
  6. 3D timeseries lasting 1.0 ms


### 2.3.1. 3x3 Grid with 50 uM pixel:

In [11]:
# Show [Ca2+] Checkpoints
t_idx_plot = np.int32(np.linspace(0, len(time)-1, 3))

ca_colorscale = [(0.0, '#00008B'), (0.2, '#6495ED'), (0.4, '#00BFFF'), 
              (0.6, '#00CED1'), (0.8, '#7FFFD4'), (1.0, '#F0FFFF')]

fig = make_subplots(rows=1, cols=3,\
              subplot_titles=(f't = {round(time[t_idx_plot[0]],2)}', f't = {round(time[t_idx_plot[1]],2)}', f't = {round(time[t_idx_plot[2]],2)}'),\
              horizontal_spacing=0.1)

simulation = allosteric_model.simulations[-2]
time = simulation['simulation'][0]
stim = simulation['simulation'][1]
stim_max = np.max(stim[-1])

for col_idx, t_idx in enumerate(t_idx_plot):
       fig.add_trace(go.Heatmap(z=stim[t_idx], colorscale=ca_colorscale, zmin=0, zmax=stim_max, showscale=False), row=1, col=col_idx+1)

fig.update_yaxes(title='[Ca2+] (uM)', row=1, col=1)
fig.update_xaxes(showticklabels=False, showline=False)
fig.update_yaxes(showticklabels=False, showline=False)
fig.show()

In [12]:
# Show state probabilities in central pixel:
data = np.column_stack((time, allosteric_model.simulations[-2]['states'][:,:,1,1]))
results = pd.DataFrame(data, columns=columns)

fig = px.line(results, x=time_label, y=allosteric_model.metadata['state_names'], title='Central pixel')
fig.update_layout(plot_bgcolor='white')
fig.update_xaxes(ticks='outside', linecolor='black')
fig.update_yaxes(title='Probability', range=[0,1], ticks='outside', linecolor='black')
fig.show()

### 2.3.2. Linearly growing 2D Gaussian

In [13]:
t_idx_plot = np.int32(np.linspace(0, len(time)-1, 3))

ca_colorscale = [(0.0, '#00008B'), (0.2, '#6495ED'), (0.4, '#00BFFF'), 
              (0.6, '#00CED1'), (0.8, '#7FFFD4'), (1.0, '#F0FFFF')]

fig = make_subplots(rows=2, cols=3,\
              subplot_titles=(f't = {round(time[t_idx_plot[0]],2)}', f't = {round(time[t_idx_plot[1]],2)}', f't = {round(time[t_idx_plot[2]],2)}', '', '', ''),\
              vertical_spacing=0.1, horizontal_spacing=0.1, row_heights=[0.5, 0.5])

simulation = allosteric_model.simulations[-1]
time = simulation['simulation'][0]
stim = simulation['simulation'][1]
stim_max = np.max(stim[-1])
prob = simulation['states'][:,-1,:,:]

for col_idx, t_idx in enumerate(t_idx_plot):
       fig.add_trace(go.Heatmap(z=stim[t_idx], colorscale=ca_colorscale, zmin=0, zmax=stim_max, showscale=False), row=1, col=col_idx+1)
       fig.add_trace(go.Heatmap(z=prob[t_idx], zmin=0, zmax=1, showscale=False), row=2, col=col_idx+1)
       fig.update_yaxes(title='[Ca2+] (uM)', row=1, col=1)
       fig.update_yaxes(title='Release Prob.', row=2, col=1)

fig.update_xaxes(showticklabels=False, showline=False)
fig.update_yaxes(showticklabels=False, showline=False)

fig.show()

### 2.3.3. Multiprocessing

Each timeseries pixel of a 3D stimulus is simulated independently. By default this is parallelised across as many processes as are available on the local system.

In [14]:
# Store and clear simulations from the model
completed_sims = allosteric_model.clear_simulations()

# Repeat Gaussian stimulation
t_end = 1 # ms
ca = 50 # uM
n_points = 1000
kernel_size = 50
time = np.linspace(0,t_end, n_points)
ca_max = 5e2 # uM
stimulus = np.asarray([gaussuian_filter(kernel_size, sigma=0.5, muu=0) * ca for ca in np.linspace(0, ca_max, n_points)])
simulations = [(time, stimulus)]

print(f"Simulation time with {completed_sims[-1]['n_processes']} processes: {completed_sims[-1]['sim_time']} s")
allosteric_model.multiprocessing = False
print(f"Running without multiprocessing...")
allosteric_model.run_simulations(simulations)
print(f"Completed. Duration: {allosteric_model.simulations[0]['sim_time']} s")

Simulation time with 20 processes: 3.1388444999975036 s
Running without multiprocessing...
Completed. Duration: 15.640170899998338 s
